<a href="https://colab.research.google.com/github/jadin101777/emcc-otp-reuse-lab/blob/main/otp_reuse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OTP Reuse Lab — EMCC Cybersecurity & Coding Club
This lab demonstrates what happens when a one-time pad (OTP) or key material
is reused between different messages.

 **Goal:** Learn how XOR and crib-dragging can reveal plaintext in reused-key scenarios.

>  Educational use only — do not apply on real systems.


In [1]:
# --- Setup ---
!pip install wordfreq --quiet

import random
import string
from wordfreq import zipf_frequency
import itertools

# Simple helpers
def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))

def is_printable(b: bytes, threshold=0.9) -> bool:
    txt = b.decode("latin1", errors="ignore")
    printable = sum(c.isprintable() or c in "\n\t" for c in txt)
    return printable / len(txt) > threshold if b else False


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00


In [2]:
# --- Generate random plaintext messages ---
# Replace these with messages related to club topics if you want variety
plaintexts = [
    b"Meet me at the north gate at 2200 hours.",
    b"The backup key is hidden under the loose tile.",
    b"Mission confirmed, proceed as planned.",
    b"The target file is named config_backup.zip.",
    b"Oh my God, they killed Kenny!."
    b"You bastards"
]

# Generate a single random key of same length (reuse vulnerability!)
key = bytes(random.randint(0, 255) for _ in range(max(len(p) for p in plaintexts)))

cts = [xor_bytes(p, key) for p in plaintexts]
print(f"Generated {len(cts)} ciphertexts with REUSED key.")
for i, c in enumerate(cts):
    print(f"c{i} =", c.hex())


Generated 5 ciphertexts with REUSED key.
c0 = 4618f56dfee421981be6e72eb671166dff2aff7ca91a3df38283431fe015a43786657ee907bdd483
c1 = 5f15f539bce827d30fe2e731bb6d166ae378e37ded1939e9c7d64c0fa555b673de2036ea1da0d4c8b486103baae7
c2 = 4614e36ab7e62a9819fda93cb7665b66f474ab64fb123fe282c7020ab307e66bd72b78e316e1
c3 = 5f15f539aae836df1fe6e73cb7785323f92bab7ae81039e3c7c04d05a64ef158d42475ed07bf89d7fd8257
c4 = 4815f177b9ec64cc12f7e72abf674574ff2aef34e81b28e29583511ea344f374c52363ea52bad7c1fb931d79


In [3]:
# --- Crib-dragging helpers ---
def crib_drag(xor_stream: bytes, crib: bytes, offset: int) -> bytes:
    frag = xor_bytes(xor_stream[offset:offset+len(crib)], crib)
    return frag

def show_recovered_fragment(crib: bytes):
    for i, (c1, c2) in enumerate(itertools.combinations(cts, 2)):
        x = xor_bytes(c1, c2)
        for off in range(0, len(x)-len(crib)+1):
            frag = crib_drag(x, crib, off)
            if is_printable(frag):
                print(f"Pair ({i}): offset {off} → {frag}")

common_bytes = [w.encode() for w in ["Meet", " at ", "north", "upload", "key", "file", "the "]]
print("Helpers loaded. Ready to test cribs.")


Helpers loaded. Ready to test cribs.


In [4]:
# --- Manual crib-dragging example ---
crib = b"Meet "
x = xor_bytes(cts[0], cts[1])

for off in range(0, len(x)-len(crib)+1):
    cand = crib_drag(x, crib, off)
    if is_printable(cand):
        print("offset", off, "→", cand)


offset 0 → b'The b'
offset 1 → b'@e16,'
offset 2 → b"M1'x&"
offset 5 → b'Ac.`$'
offset 6 → b'K.qp '
offset 8 → b'Yaek-'
offset 9 → b'Iezy<'
offset 10 → b'Mzhh '
offset 11 → b"Rhyt'"
offset 12 → b'@yes<'
offset 13 → b'Qebhr'
offset 14 → b'Mby&<'
offset 15 → b'Jy7h!'
offset 16 → b'Q7yud'
offset 18 → b'Qd!w$'
offset 19 → b'L!fp:'
offset 20 → b'\tfane'
offset 23 → b'W 0{0'
offset 26 → b'Bu 42'
offset 27 → b'] %fd'
offset 30 → b'_!=1h'
offset 31 → b'\t= <#'


In [5]:
# --- Verify a crib by deriving key fragment ---
offset = 0  # replace with your discovered offset
crib = b"Meet "  # same crib

k_frag = xor_bytes(cts[0][offset:offset+len(crib)], crib)
recovered = xor_bytes(cts[1][offset:offset+len(crib)], k_frag)

print("Recovered:", recovered)


Recovered: b'The b'


In [6]:
# --- Automated crib search ---
def automated_crib_search(i, j, crib_list, show_hits=15):
    c1, c2 = cts[i], cts[j]
    x = xor_bytes(c1, c2)
    found = []
    for crib in crib_list:
        for off in range(0, len(x)-len(crib)+1):
            frag = crib_drag(x, crib, off)
            if is_printable(frag):
                found.append((crib, off, frag))
    for c, o, f in found[:show_hits]:
        print(f"Crib {c!r} at offset {o} → {f!r}")

automated_crib_search(0, 1, common_bytes, show_hits=20)


Crib b'Meet' at offset 0 → b'The '
Crib b'Meet' at offset 1 → b'@e16'
Crib b'Meet' at offset 2 → b"M1'x"
Crib b'Meet' at offset 5 → b'Ac.`'
Crib b'Meet' at offset 6 → b'K.qp'
Crib b'Meet' at offset 8 → b'Yaek'
Crib b'Meet' at offset 9 → b'Iezy'
Crib b'Meet' at offset 10 → b'Mzhh'
Crib b'Meet' at offset 11 → b'Rhyt'
Crib b'Meet' at offset 12 → b'@yes'
Crib b'Meet' at offset 13 → b'Qebh'
Crib b'Meet' at offset 14 → b'Mby&'
Crib b'Meet' at offset 15 → b'Jy7h'
Crib b'Meet' at offset 16 → b'Q7yu'
Crib b'Meet' at offset 18 → b'Qd!w'
Crib b'Meet' at offset 19 → b'L!fp'
Crib b'Meet' at offset 20 → b'\tfan'
Crib b'Meet' at offset 23 → b'W 0{'
Crib b'Meet' at offset 26 → b'Bu 4'
Crib b'Meet' at offset 27 → b'] %f'


## Challenge
Try to:
- Recover at least one readable plaintext fragment.
- Identify which ciphertext pairs share the most overlap.
- Modify `plaintexts` list and see how results change.
- (Optional) Write a function that reconstructs a full sentence.


## Discussion & Mitigation
- OTP reuse makes ciphertexts mathematically linked (no secrecy).
- Always use unique keys, IVs, or nonces for every message.
- Randomness quality and operational discipline are as critical as algorithms.


---
**EMCC Cybersecurity & Coding Club — Peer-Led Learning**  

